<a href="https://colab.research.google.com/github/JA-Bar/multimodal-sentiment-analysis/blob/kevin/Glove_dataset_hugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'emotion')

In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official
!pip install -q sklearn

In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

In [ ]:
import tensorflow as tf
import tensorflow_text as text


from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,SimpleRNN,GRU,RNN
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

from numpy import array
from numpy import asarray
from numpy import zeros


import tensorflow_hub as hub
import tensorflow as tf
from official.nlp import optimization 
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

In [ ]:
def adding_new_data(dataframe_input):
  place=0
  for sent in dataframe_input['sentiment']:
    if sent==0:
      dataframe_input['sadness'][place]=1
    elif sent==1:
      dataframe_input['joy'][place]=1
    elif sent==2:
      dataframe_input['love'][place]=1
    elif sent==3:
      dataframe_input['anger'][place]=1
    elif sent==4:
      dataframe_input['fear'][place]=1
    elif sent==5:
      dataframe_input['unsolve'][place]=1
    place += 1
  dataframe_input['sadness']=dataframe_input['sadness'].replace(np.nan, 0)
  dataframe_input['joy']=dataframe_input['joy'].replace(np.nan, 0)
  dataframe_input['love']=dataframe_input['love'].replace(np.nan, 0)
  dataframe_input['anger']=dataframe_input['anger'].replace(np.nan, 0)
  dataframe_input['fear']=dataframe_input['fear'].replace(np.nan, 0)
  dataframe_input['unsolve']=dataframe_input['unsolve'].replace(np.nan,0)
  return dataframe_input

In [ ]:
def data (dataset_raw,x):
  data=dataset_raw[x]
  labels=data['label']
  text=data['text']
  text=np.array([text])
  labels=np.array([labels])
  data4=pd.DataFrame([text[0],labels[0]])
  data4=data4.T
  data4.columns=['text','sentiment']
  data4["sadness"] = np.nan
  data4["joy"] = np.nan
  data4["love"] = np.nan
  data4["anger"] = np.nan
  data4["fear"] = np.nan
  data4["unsolve"] = np.nan
  data4=adding_new_data(data4)
  return data4

In [ ]:
train=data (dataset,'train')
test=data (dataset,'test')
validation=data (dataset,'validation')

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')
validation.to_csv('validation.csv')

In [ ]:
train.head(10)

In [ ]:
print(train.size)

In [ ]:
concat_data=[train,test,validation]
train = pd.concat(concat_data)

In [ ]:
train

In [ ]:
print(train.size)

In [ ]:
place=168
filter = train["text"] != ""
train = train[filter]
train = train.dropna()
print('text: ' + train["text"][place])
print("label:" + str(train["sentiment"][place]))

In [ ]:
#train_labels = train[["sadness", "joy", "love",'anger','fear','unsolve']]
train_labels = train[["sadness", "joy"]]
#train_labels = train[["joy","sadness"]]
classes=train_labels.shape
classes=classes[1]
print(classes)
train_labels.head()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size
train_labels.sum(axis=0).plot.bar()

In [ ]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [ ]:
X = []
sentences = list(train["text"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = train_labels.values

In [ ]:
X[1:10]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(y_train)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
print(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = None

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(X_train[1])

In [ ]:
print(X_train[2])

In [ ]:
print(X_train[3])

In [ ]:
embeddings_dictionary = dict()
glove_file = open('/content/gdrive/MyDrive/Clasificador/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
#print(embedding_matrix)

In [ ]:
deep_inputs = Input(shape=(None,))
print(deep_inputs)
net = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False,mask_zero=True,name = 'embeddings')(deep_inputs)
print(net)
net = LSTM(200, return_sequences=True,name='lstm_layer1')(net)
net = SimpleRNN(200, return_sequences=True,name='RNN')(net)
net = GRU(100, return_sequences=True,name='GRU')(net)
net = LSTM(150,name='lstm_layer2')(net)
#net = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation=None, kernel_initializer='he_uniform')(net)
#net = tf.keras.layers.MaxPooling1D(3)(net)
#net = tf.keras.layers.GlobalMaxPool1D()(net)
#net = tf.keras.layers.BatchNormalization()(net)
#net=tf.keras.layers.Dropout(0.2)(net)
#net = Dense(25,activation=None)(net)
#net = Dense(12,activation=None)(net)
#net = Dense(6,activation=None)(net)
net = Dense(classes,activation='sigmoid')(net)
model2= Model(inputs=deep_inputs, outputs=net)
opt = tf.optimizers.Adam(learning_rate=0.01)
model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])
print(model2.summary())

In [ ]:
history = model2.fit(X_train, y_train, batch_size=16, epochs=50, verbose=1, validation_split=0.2)

In [ ]:
score = model2.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
def convert2embeding (word):
  X = []
  sentences = list(word)
  #print(sentences)
  for sen in sentences:
    X.append(preprocess_text(sen))
  #print(X)
  token_word=tokenizer.texts_to_sequences(X[0])
  #print(token_word)
  final=[]
  for tok in token_word:
    try:
      final.append(tok[0])
    except:
      final.append(0)
  #print(final)
  maxlen = 50
  final_word = pad_sequences([final], padding='post', maxlen=maxlen)
  #print(final_word)
  return final_word
word2='Beatiful world baby'
x=convert2embeding ([word2])
print(x)

In [ ]:
path='/content/gdrive/MyDrive/Clasificador/glove_dataset_hugging_face.h5'
model2.save(path)
new_model = keras.models.load_model(path)

In [ ]:
["sadness", "joy", "love",'anger','fear','unsolve']

In [ ]:
#Many
y='beatiful world baby'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,' ',a)
y='hate you'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='very bored'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='i was so happy back then'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='i am so happy'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='is so scary'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='i hate my life'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='i love you'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='let´s party all night!!!!'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='lets have some fun baby'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='i want to kill myself'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y='fucking bitch'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y= 'i am so sad'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)
y= 'i used to be happy'
x2=convert2embeding([y])
a = model2.predict(x2)
print(y,'---- ',a)


In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')